### Data Collection for Database
1. Stock Data: Stock List [NASDAQ-100], General Information, Fundamental data, Yearly Data, Intraday Data, Analyst Ratings]
2. Sentiment Data: Using Vader for each stock

In [2]:
# imports
import yfinance as yf
import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/opt/anaconda3/lib/python3.9/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
ticker = 'BTC-USD'
data = yf.download(ticker, period="20y", interval="1d")
data.to_csv('stock_prices.csv')

[*********************100%***********************]  1 of 1 completed


In [ ]:
# stock list
def fetch_stock_list():
    url = "https://api.nasdaq.com/api/quote/list-type/nasdaq100"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    response = requests.get(url, headers=headers)
    data = response.json()
    stock_list = [item["symbol"] for item in data["data"]["data"]["rows"]]
    return stock_list

In [26]:
# name, sector, industry, description
def stock_general_info(ticker):

    # name, sector, industry - yahoo finance
    url = f"https://finance.yahoo.com/quote/{ticker}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    name = soup.find("h1", class_="D(ib)").text
    sector = soup.find("span", text="Sector(s)").find_next_sibling("span").text
    industry = soup.find("span", text="Industry").find_next_sibling("span").text

    # general info - marketwatch
    url = f"https://www.marketwatch.com/investing/stock/{symbol}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    description = soup.find("p", class_="description__text").text.strip()
    

    return (name,sector,industry,description)

In [30]:
# yearly prices
def yearly_prices(ticker):
    data = yf.download(ticker, period="1y", interval="1d")
    return data[['Adj Close']].reset_index()

In [ ]:
# intraday prices
def intraday_prices(ticker):
    data = yf.download(ticker, period="1d", interval="2m")
    return data[['Adj Close']].reset_index()

In [28]:
# analyst ratings
def analyst_ratings(ticker):
    stock = yf.Ticker(ticker)
    if (stock.info['targetMeanPrice']):
        targetPrice = {
            "category": "Target Price",
            "Low": stock.info['targetLowPrice'],
            "Mean": stock.info['targetMeanPrice'],
            "High": stock.info['targetHighPrice']
        }
        return targetPrice
    else:
        return None

In [85]:
# sentiment analysis
def find_sentiment(ticker):
    url = f'https://finviz.com/quote.ashx?t={ticker}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, features='html.parser')
    news_table = soup.find(id='news-table')

    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = []
    for row in news_table.findAll('tr'):
        date_data = row.td.text.split(' ')
        if len(date_data) == 1:
            date = date
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]
        headline = row.a.text
        print(headline)
        sentiment_score = analyzer.polarity_scores(headline)['compound']
        sentiment_scores.append(sentiment_score)


    overall_sentiment_score = sum(sentiment_scores) / len(sentiment_scores)
    return(overall_sentiment_score)

